## Hashing trick + many features from for look-alike portal application completed


In [2]:
#Config
from pyspark import SparkConf, SparkContext, HiveContext
#import re
#import numpy as np
#import pandas as pd
#import datetime
#from pyspark.mllib.regression import LabeledPoint
#from pyspark.mllib.feature import HashingTF
#from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
#import scipy.sparse as sps
#from pyspark.mllib.linalg import Vectors
#import sklearn
try:
    sc.stop()
except NameError: pass


conf = SparkConf().set("spark.executor.instances", 8).set("spark.driver.maxResultSize", "8g")
sc = SparkContext(conf=conf)
hc = HiveContext(sc)

import pandas as pd
import numpy as np
import os
import sklearn
import re
import xgboost as xgb

In [17]:
cols = [e for e in hc.sql('select * from user_kposminin.la_max_score_and_hash_20161013 limit 1').collect()[0].asDict().keys() if e != 'features_set']


['good_tcs_urlfr_visited_cnt',
 'q50_tcs_score',
 'sum_duration',
 'max_urlfr_avg_hour',
 'avg_duration',
 'max_tcs_score',
 'hour_std',
 'id',
 'mobile_share',
 'q75_tcs_score',
 'very_good_tcs_urlfr_visited_cnt',
 'cnt',
 'max_urlfr_hits_cnt',
 'min_avg_hour',
 'social_share',
 'avg_avg_hour',
 'label',
 'avg_work_hours_tcs_score',
 'max_work_hours_tcs_score',
 'max_urlfr_cnt_total',
 'min_tcs_score',
 'q25_tcs_score',
 'log_avg_exp_tsc_score',
 'vk_share',
 'max_avg_hour',
 'q90_tcs_score',
 'hour_cnt',
 'avg_tcs_score',
 'hits',
 'first_day',
 'sum_tcs_score',
 'work_hours_hits',
 'emailru']

In [20]:
df_train = hc.sql('select ' + ','.join(cols) + ' from user_kposminin.la_max_score_and_hash_20161013').toPandas()

print(len(train))
train_hash_pred = [float(e) for e in open('/home/k.p.osminin/external_hdfs/pred_20161013.txt','r')]
print(len(train_hash_pred))

756555
756555


In [21]:
df_train['hash_score'] = train_hash_pred

In [25]:
feat_cols = [f for f in df_train.columns if not f in ['label','first_day','id']]


In [97]:
df_train.columns

Index([u'good_tcs_urlfr_visited_cnt', u'q50_tcs_score', u'sum_duration',
       u'max_urlfr_avg_hour', u'avg_duration', u'max_tcs_score', u'hour_std',
       u'id', u'mobile_share', u'q75_tcs_score',
       u'very_good_tcs_urlfr_visited_cnt', u'cnt', u'max_urlfr_hits_cnt',
       u'min_avg_hour', u'social_share', u'avg_avg_hour', u'label',
       u'avg_work_hours_tcs_score', u'max_work_hours_tcs_score',
       u'max_urlfr_cnt_total', u'min_tcs_score', u'q25_tcs_score',
       u'log_avg_exp_tsc_score', u'vk_share', u'max_avg_hour',
       u'q90_tcs_score', u'hour_cnt', u'avg_tcs_score', u'hits', u'first_day',
       u'sum_tcs_score', u'work_hours_hits', u'emailru', u'hash_score',
       u'xgboost_pred'],
      dtype='object')

In [26]:
dtrain = xgb.DMatrix( df_train[feat_cols], label=df_train.label)

In [27]:
param = {'max_depth':10, 'eta':1, 'gamma' : 0.5, 'silent':1, 'objective':'binary:logistic','eval_metric' : "auc",
         'scale_pos_weight' : 1, 'num_round' : 10 }

bst = xgb.train(param, dtrain)

# make prediction
df_train['xgboost_pred'] = bst.predict(dtrain)

In [31]:

print('{0} AUCROC train: tcs_max_score {1:.6f}. ha_score {2:.6f}. xgboost(tcs_max,ha_sc) {3:.6f}'.format(
               'XGBoost',
                sklearn.metrics.roc_auc_score(
                  y_true = df_train.label, 
                  y_score = df_train.max_tcs_score.fillna(-1000)
                ),
                sklearn.metrics.roc_auc_score(
                  y_true = df_train.label, 
                  y_score = df_train.hash_score.fillna(-1000)
                ),                                              
               sklearn.metrics.roc_auc_score(
                  y_true = df_train.label, 
                  y_score = df_train.xgboost_pred.fillna(-1000)
                ),                                              
))

XGBoost AUCROC train: tcs_max_score 0.855062. ha_score 0.790676. xgboost(tcs_max,ha_sc) 0.964526


In [36]:
df_test = hc.sql('select ' + ','.join(cols) + ' from user_kposminin.la_max_score_and_hash_20161020').toPandas()

print(len(df_test))
test_hash_pred = [float(e) for e in open('/home/k.p.osminin/external_hdfs/pred_20161020.txt','r')]
print(len(test_hash_pred))
df_test['hash_score'] = test_hash_pred

KeyboardInterrupt: 

In [148]:
#a = open('./external_hdfs/ms_hs_pred_20161020.txt','r').readline()
#indxs = [(cols+['hash_score']).index(c) for c in feat_cols]
label_idx=16
hash_idx = -1
max_sc_indx = 5
#[[float(l.strip().replace(' ','\t').split('\t')[i]) for i in indxs]  for l in [a]]
#f = open('./external_hdfs/ms_hs_pred_20161020.txt','r')
#batch = [l.strip().replace(' ','\t').split('\t') for l in itert.islice(f, 1000)]
#batch[2][0].isdigit()
#xgb.DMatrix(pd.DataFrame(batch1,columns = (df_train.columns[:-1])))
#dtrain = xgb.DMatrix( df_train[feat_cols], label=df_train.label)
#batch1 = [[(float(l[i]) if l[i].isdigit() else -99) if l[i] else -99 for i in range(34)] for l in batch]
#df_train.iloc[0]
#batch1 = [[(float(l[i]) if l[i].isdigit() else -99) if l[i] else -99 for i in indxs] for l in batch]
#bst.predict(xgb.DMatrix( pd.DataFrame(batch1, columns = feat_cols), label=label))


IndexError: list index out of range

In [149]:
import itertools as itert
res = [
    [], #label
    [], #max_tcs_score
    [], #hash score
    []  # full xgb score
    
]
#bst.save_model('xgb_bst.model')
batch_size = 1 * 10 ** 6
cnt=0

def conv_float(s):
    try:
        return float(s)
    except:
        return -99

def conv_label(s):
    try:
        return int(s)
    except:
        return 0

with open('./external_hdfs/ms_hs_pred_20161020.txt','r') as f:
    while True:
        batch = [l.strip().replace(' ','\t').split('\t') for l in itert.islice(f, batch_size)]
        if not batch:
            break
        #batch1 = [[(float(l[i]) if l[i].isdigit() else -99) if l[i] else -99 for i in range(34)] for l in batch]
        #batch1 = [[(float(l[i]) if l[i].isdigit() else -99) if l[i] else -99 for i in indxs] for l in batch]
        #label = [(int(l[label_idx]) if l[label_idx].isdigit() else 0) if l[label_idx] else 0 for l in batch]
        #max_sc = [(float(l[max_sc_indx]) if l[max_sc_indx].isdigit() else -99) if l[max_sc_indx] else -99 for l in batch]
        #ha_sc = [(float(l[hash_idx]) if l[hash_idx].isdigit() else -99) if l[hash_idx] else -99 for l in batch]
        batch1 = [[conv_float(l[i]) for i in indxs] for l in batch]
        label = [conv_label(l[label_idx]) for l in batch]
        max_sc = [conv_float(l[max_sc_indx]) for l in batch]
        ha_sc = [conv_float(l[hash_idx]) for l in batch]
        xgb_sc = bst.predict(xgb.DMatrix( pd.DataFrame(batch1, columns = feat_cols), label=label))
        res[0] = res[0] + label
        res[1] = res[1] + max_sc
        res[2] = res[2] + ha_sc
        res[3] = res[3] + xgb_sc.tolist()
        print(cnt)
        cnt += 1
        # process next_n_lines

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193


In [150]:
import cPickle
cPickle.dump(res,open('./data/res.pck','w'))
#df_test['xgboost_pred'] = bst.predict(xgb.DMatrix( df_test[feat_cols], label=df_test.label))

In [151]:

print('{0} AUCROC test: tcs_max_score {1:.6f}.'.format(
               ' ',
                sklearn.metrics.roc_auc_score(
                  y_true = res[0], 
                  y_score = res[1]
                )
    ))
print('{0} AUCROC test: hash_score {1:.6f}.'.format(
               ' ',
                sklearn.metrics.roc_auc_score(
                  y_true = res[0], 
                  y_score = res[2]
                )
    ))
print('{0} AUCROC test: full xgb score {1:.6f}.'.format(
               ' ',
                sklearn.metrics.roc_auc_score(
                  y_true = res[0], 
                  y_score = res[3]
                )
    ))

  AUCROC test: tcs_max_score 0.843219.
  AUCROC test: hash_score 0.786736.
  AUCROC test: full xgb score 0.810321.


In [ ]:
print('{0} AUCROC test: tcs_max_score {1:.6f}. ha_score {2:.6f}. xgboost(tcs_max,ha_sc) {3:.6f}'.format(
               'XGBoost',
                sklearn.metrics.roc_auc_score(
                  y_true = df_test.label, 
                  y_score = df_test.tcs_maxs
                ),
                sklearn.metrics.roc_auc_score(
                  y_true = df_test.label, 
                  y_score = df_test.hash_score
                ),                                              
               sklearn.metrics.roc_auc_score(
                  y_true = df_test.label, 
                  y_score = df_test.xgboost_pred
                ),                                              
))

,good_tcs_urlfr_visited_cnt,q50_tcs_score,sum_duration,max_urlfr_avg_hour,avg_duration,max_tcs_score,hour_std,id,mobile_share,q75_tcs_score,...,q90_tcs_score,hour_cnt,avg_tcs_score,hits,first_day,sum_tcs_score,work_hours_hits,emailru,hash_score,xgboost_pred
0,0,-14.456675,0,7,0.000000,-14.456675,0.000000,001A2FF44D018EBF2929,0.000000,-14.456675,...,-14.456675,1,-14.456675,2,0,-14.456675,0,0,0.002222,0.000018
1,0,-10.091718,6235,2,215.000000,-10.037920,0.000000,007B4CCA00EAF2A795EE,0.000000,-10.047165,...,-10.044210,1,-10.101387,150,0,-70.709708,0,0,0.002620,0.005007
2,0,-9.659526,327945,10,2429.222222,-9.191184,1.948585,00B42817F3C590D2565D,0.000000,-9.429954,...,-9.303338,8,-9.771383,1041,0,-644.911290,849,0,0.011591,0.004211
3,0,-17.918807,34840,19,5806.666667,-16.588208,2.285218,00E0492156AACC8DA69C,0.000000,-16.933061,...,-16.726149,5,-17.475274,12,0,-52.425822,9,0,0.002222,0.000037
4,0,-10.293265,144670,8,2836.666667,-9.428934,3.391760,00E43011D5AEB186ECE0,0.000000,-10.018047,...,-9.733399,8,-10.444749,231,0,-261.118715,180,0,0.005100,0.000667
5,0,-15.083193,3,20,0.428571,-12.605746,0.000000,00EC80FB0EDE6BC9A42E,0.000000,-12.930917,...,-12.614896,1,-14.342381,11,0,-71.711905,11,0,0.002222,0.000018
6,0,-15.484961,0,4,0.000000,-14.446990,0.000000,0122F396B0B298DB4048,0.000000,-14.965976,...,-14.654584,1,-14.965976,3,0,-29.931952,0,0,0.002222,0.000018
7,0,-11.540443,21,16,7.000000,-11.125556,0.000000,012A6DBDF71D8828F2C0,1.000000,-11.152256,...,-11.136236,1,-11.402147,6,0,-34.206442,6,0,0.002667,0.001609
8,11,-8.844310,1907,9,90.809524,-8.078934,0.436436,013C87BCB434EC53C23F,0.000000,-8.306012,...,-8.208297,3,-8.835910,100,0,-123.702736,96,0,0.006344,0.036306
9,0,-11.050843,3715,1,285.769231,-10.495304,0.000000,01D478C1F2239D5733A7,0.000000,-10.740574,...,-10.600701,1,-10.926176,24,0,-98.335583,0,0,0.002222,0.004204


In [18]:
df_train.loc[:,'tcs_maxs_bin'] = pd.cut(
    df_train['tcs_maxs'], 
    bins = np.unique(df_train['tcs_maxs'].quantile(np.arange(0,1.05,0.05)).values), 
    labels  = False).values

df_train.loc[:,'hash_score_bin'] = pd.cut(
    df_train['hash_score'], 
    bins = np.unique(df_train['hash_score'].quantile(np.arange(0,1.05,0.05)).values), 
    labels  = False).values

In [ ]:
#tab = df_train.groupby(['tcs_maxs_bin','hash_score_bin'])['label'].agg([np.mean,np.size]).unstack() 
#tab['size']

In [ ]:

print('{0} AUCROC train: tcs_max_score {1:.6f}. ha_score {2:.6f}. xgboost(tcs_max,ha_sc) {3:.6f}'.format(
               'RandomForest',
                sklearn.metrics.roc_auc_score(
                  y_true = df_train.label, 
                  y_score = [e[1] for e in clfRF.predict_proba(df_train[['tcs_maxs','hash_score']])]
                ),
                sklearn.metrics.roc_auc_score(
                  y_true = df_test.label, 
                  y_score = [e[1] for e in clfRF.predict_proba(df_test[['tcs_maxs','hash_score']])]
                ),
))


In [47]:
df_train_top = df_train[df_train['tcs_maxs'] > df_train['tcs_maxs'].quantile(0.95)]

In [48]:
df_train_top.loc[:,'hash_score_bin'] = pd.cut(
    df_train_top['hash_score'], 
    bins = np.unique(df_train_top['hash_score'].quantile(np.arange(0,1.01,0.1)).values), 
    labels  = False).values

In [49]:
df_train_top.groupby('hash_score_bin')['label'].agg([np.mean,np.size])

,mean,size
hash_score_bin,,
0.0,0.001578,46896
1.0,0.002484,27779
2.0,0.002057,37428
3.0,0.002604,37244
4.0,0.003053,37337
5.0,0.003294,37339
6.0,0.003884,37335
7.0,0.004794,37337
8.0,0.006187,37337


In [64]:
df_train_sampled = df_train[(df_train.label == 1) | (np.random.rand(len(df_train)) < 0.1)]

In [68]:
import sklearn.ensemble
from sklearn import svm
clf = {
   # 'LinSVC': sklearn.svm.LinearSVC(penalty='l1', loss='squared_hinge',C=1.0, class_weight='auto', max_iter=1000),
   #'SVC': sklearn.svm.SVC(probability = True,max_iter = 100),
    'LogRegr': sklearn.linear_model.LogisticRegression(),
    'RandomForest': sklearn.ensemble.RandomForestClassifier(max_depth = 5,n_estimators = 50),
   # 'GBM': sklearn.ensemble.GradientBoostingClassifier(n_estimators = 400)
    
}
for m in clf:
    clf[m].fit(X = df_train[['tcs_maxs','hash_score']],y = df_train['label'])
    print('{0} AUCROC: {1}'.format(m,sklearn.metrics.roc_auc_score(
                y_true = df_test['label'], 
                y_score = [e[1] for e in clf[m].predict_proba(df_test[['tcs_maxs','hash_score']])]
    )))

RandomForest AUCROC: 0.858583009817
LogRegr AUCROC: 0.853759819568



#### RandomForest Немного лучше max_tcs_score на тесте.

In [110]:
#a = np_utils.normalize(df_train[['tcs_maxs','hash_score']])
from sklearn import preprocessing
import numpy as np
X = np.array([[ 1., -1.,  2.,11],
               [ 2.,  0.,  0.,13],
               [ 0.,  1., -1.,10]])
scaler = preprocessing.StandardScaler().fit(df_train_sampled[['tcs_maxs','hash_score']])
X, y = scaler.transform(df_train_sampled[['tcs_maxs','hash_score']]), df_train_sampled['label']
Xt, yt = scaler.transform(df_test[['tcs_maxs','hash_score']]), df_test['label']


Y_train = np_utils.to_categorical(df_train_sampled['label'].values, 2) # One-hot encode the labels
#Y_test = np_utils.to_categorical(df_test['label'], 2) # One-hot encode the labels

In [216]:
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Dense # the two types of neural network layer we will be using
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values


batch_size = 256
num_epochs = 15 # we iterate twenty times over the entire training set
hidden_size_1 = 32 # there will be 16 neurons in first layer
hidden_size_2 = 16 


inp = Input(shape=(2,)) # Our input is a 1D vector of size 784
hidden_1 = Dense(hidden_size_1, activation='relu')(inp) # First hidden ReLU layer
hidden_2 = Dense(hidden_size_2, activation='relu')(hidden_1) # Second hidden ReLU layer
out = Dense(2, activation='softmax')(hidden_2) # Output softmax layer

model = Model(input=inp, output=out) # To define a model, just specify its input and output layers

In [217]:
model.compile(loss='binary_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

In [218]:
model.fit(X, Y_train, # Train the model using the training set...
          batch_size=batch_size, nb_epoch=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation
#model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Train on 675550 samples, validate on 75062 samples
Epoch 1/15
675550/675550 [==============================] - 6s - loss: 0.0438 - acc: 0.9919 - val_loss: 0.0288 - val_acc: 0.9942
Epoch 2/15
675550/675550 [==============================] - 6s - loss: 0.0277 - acc: 0.9944 - val_loss: 0.0287 - val_acc: 0.9942
Epoch 3/15
675550/675550 [==============================] - 6s - loss: 0.0277 - acc: 0.9944 - val_loss: 0.0291 - val_acc: 0.9939
Epoch 4/15
675550/675550 [==============================] - 6s - loss: 0.0277 - acc: 0.9944 - val_loss: 0.0288 - val_acc: 0.9942
Epoch 5/15
675550/675550 [==============================] - 7s - loss: 0.0277 - acc: 0.9945 - val_loss: 0.0291 - val_acc: 0.9940
Epoch 6/15
675550/675550 [==============================] - 8s - loss: 0.0276 - acc: 0.9945 - val_loss: 0.0289 - val_acc: 0.9943
Epoch 7/15
675550/675550 [==============================] - 6s - loss: 0.0276 - acc: 0.9945 - val_loss: 0.0288 - val_acc: 0.9942
Epoch 8/15
675550/675550 [====================

In [219]:
    print('{0} AUCROC: {1}'.format('Keras train',sklearn.metrics.roc_auc_score(
                y_true = y,
                y_score =[e[1] for e in model.predict(X)]
    )))


Keras train AUCROC: 0.868020784375


In [220]:
    print('{0} AUCROC: {1}'.format('Keras test',sklearn.metrics.roc_auc_score(
                y_true = yt,
                y_score = [e[1] for e in model.predict(Xt)]
    )))

#model.predict(Xt)

Keras test AUCROC: 0.86265400094


In [138]:
from sklearn.ensemble import AdaBoostClassifier

cl = AdaBoostClassifier()
cl.fit(df_train[['tcs_maxs','hash_score']], df_train['label'])

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [140]:
print('{0} AUCROC train: {1:.6f}. test: {2:.6f}'.format(
               'AdaBoost',
                sklearn.metrics.roc_auc_score(
                  y_true = df_train.label, 
                  y_score = cl.predict_proba(df_train[['tcs_maxs','hash_score']])[:,1]
                ),
                sklearn.metrics.roc_auc_score(
                  y_true = df_test.label, 
                  y_score = cl.predict_proba(df_test[['tcs_maxs','hash_score']])[:,1]
                ),
))


AdaBoost AUCROC train: 0.869603. test: 0.861106


In [156]:
from sklearn.ensemble import AdaBoostClassifier
print('Target class is first day.')
clAB1 = AdaBoostClassifier()
clAB1.fit(df_train[['tcs_maxs','hash_score']], df_train['first_day'])
print('{0} AUCROC train: {1:.6f}. test: {2:.6f}'.format(
               'AdaBoost',
                sklearn.metrics.roc_auc_score(
                  y_true = df_train['first_day'], 
                  y_score = clAB1.predict_proba(df_train[['tcs_maxs','hash_score']])[:,1]
                ),
                sklearn.metrics.roc_auc_score(
                  y_true = df_test['first_day'], 
                  y_score = clAB1.predict_proba(df_test[['tcs_maxs','hash_score']])[:,1]
                ),
)) 

AdaBoost AUCROC train: 0.877821. test: 0.868445


In [184]:
print('{:<30}  {:<10}  {:<10}  {:<10}'.format('Sample','label sum','count','label mean'))
print('{:<30}  {:<10}  {:<10}  {:<10.4f}'.format('Full train sample', df_train.label.sum(),df_train.label.count(),df_train.label.mean()))
for q in [0.7,0.8,0.9,0.95,0.98,0.99]:
    df1 = df_train[(df_train['tcs_maxs'] > df_train['tcs_maxs'].quantile(q)) | (df_train['hash_score'] > df_train['hash_score'].quantile(q))]
    print('{:<30}  {:<10}  {:<10}  {:<10.4f}'.format('tcs_maxs >q{0} or ha >q{0}'.format(q), df1.label.sum(),df1.label.count(),df1.label.mean()))
    df1 = df_train[(df_train['tcs_maxs'] > df_train['tcs_maxs'].quantile(q)) & (df_train['hash_score'] > df_train['hash_score'].quantile(q))]
    print('{:<30}  {:<10}  {:<10}  {:<10.4f}'.format('tcs_maxs >q{0} and ha >q{0}'.format(q), df1.label.sum(),df1.label.count(),df1.label.mean()))


#df_train_top1.label.sum(),df_train_top1.label.mean()

Sample                          label sum   count       label mean
Full train sample               4340        7468151     0.0006    
tcs_maxs >q0.7 or ha >q0.7      3879        2825140     0.0014    
tcs_maxs >q0.7 and ha >q0.7     3187        1552765     0.0021    
tcs_maxs >q0.8 or ha >q0.8      3530        2029848     0.0017    
tcs_maxs >q0.8 and ha >q0.8     2630        957364      0.0027    
tcs_maxs >q0.9 or ha >q0.9      2979        1123401     0.0027    
tcs_maxs >q0.9 and ha >q0.9     1846        364596      0.0051    
tcs_maxs >q0.95 or ha >q0.95    2338        601552      0.0039    
tcs_maxs >q0.95 and ha >q0.95   1279        145225      0.0088    
tcs_maxs >q0.98 or ha >q0.98    1673        254736      0.0066    
tcs_maxs >q0.98 and ha >q0.98   818         43970       0.0186    
tcs_maxs >q0.99 or ha >q0.99    1274        134312      0.0095    
tcs_maxs >q0.99 and ha >q0.99   601         14978       0.0401    


In [187]:
print('{:<30}  {:<10}  {:<10}  {:<10}'.format('Sample','label sum','count','label mean'))
print('{:<30}  {:<10}  {:<10}  {:<10.4f}'.format('Full train sample', df_train.label.sum(),df_train.label.count(),df_train.label.mean()))
for q in [0.7,0.8,0.9,0.95,0.98,0.99,0.995,0.999]:
    df1 = df_train[(df_train['tcs_maxs'] > df_train['tcs_maxs'].quantile(q))]
    print('{:<30}  {:<10}  {:<10}  {:<10.4f}'.format('tcs_maxs >q{0}'.format(q), df1.label.sum(),df1.label.count(),df1.label.mean()))


Sample                          label sum   count       label mean
Full train sample               4340        7468151     0.0006    
tcs_maxs >q0.7                  3555        2138254     0.0017    
tcs_maxs >q0.8                  3117        1493582     0.0021    
tcs_maxs >q0.9                  2418        741182      0.0033    
tcs_maxs >q0.95                 1853        373370      0.0050    
tcs_maxs >q0.98                 1303        149343      0.0087    
tcs_maxs >q0.99                 986         74608       0.0132    
tcs_maxs >q0.995                826         36200       0.0228    
tcs_maxs >q0.999                652         7477        0.0872    


In [191]:
print('{:<30}  {:<10}  {:<10}  {:<10}'.format('Sample','label sum','count','label mean'))
print('{:<30}  {:<10}  {:<10}  {:<10.4f}'.format('Full train sample', df_train.label.sum(),df_train.label.count(),df_train.label.mean()))
for q in [0.7,0.8,0.9,0.95,0.98,0.99,0.995,0.999]:
    df1 = df_train[df_train['hash_score'] > df_train['hash_score'].quantile(q)]
    print('{:<30}  {:<10}  {:<10}  {:<10.4f}'.format('ha >q{0}'.format(q), df1.label.sum(),df1.label.count(),df1.label.mean()))


Sample                          label sum   count       label mean
Full train sample               4340        7468151     0.0006    
ha >q0.7                        3511        2239651     0.0016    
ha >q0.8                        3043        1493630     0.0020    
ha >q0.9                        2407        746815      0.0032    
ha >q0.95                       1764        373407      0.0047    
ha >q0.98                       1188        149363      0.0080    
ha >q0.99                       889         74682       0.0119    
ha >q0.995                      694         37341       0.0186    
ha >q0.999                      417         7469        0.0558    
